# ONNX to TVM

In [1]:
# 导入onnx，转换成*.so动态库
import onnx
import time
import tvm
import numpy as np
import tvm.relay as relay
from PIL import Image

In [2]:
#开始同样是读取.onnx模型
onnx_model = onnx.load('../models/yolov4.onnx')  # 导入模型

img = Image.open("street.jpg")
img = img.resize((416, 416))
img = np.array(img, dtype=np.float32)
img /= 255.0
img = img.transpose((2, 0, 1))
img = np.expand_dims(img, axis=0)

# 这里首先在PC的CPU上进行测试 所以使用LLVM进行导出
# target = tvm.target.create('llvm') # x86
target = tvm.target.Target('llvm') # x86
# target = tvm.target.arm_cpu("rasp3b") # raspi
# target = 'llvm'


input_name = "input_0"  # 注意这里为之前导出onnx模型中的模型的输入id，这里为0
shape_dict = {input_name: img.shape}
# 利用Relay中的onnx前端读取我们导出的onnx模型
sym, params = relay.frontend.from_onnx(onnx_model, shape_dict)

# 这里利用TVM构建出优化后模型的信息
with relay.build_config(opt_level=2):
    graph, lib, params = relay.build_module.build(sym, target, params=params)
    

    
dtype = 'float32'
from tvm.contrib import graph_runtime

# 下面的函数导出我们需要的动态链接库 地址可以自己定义
print("Output model files")
libpath = "../models/yolov4_pc.so"
lib.export_library(libpath)

# 下面的函数导出我们神经网络的结构，使用json文件保存
graph_json_path = "../models/yolov4_pc.json"
with open(graph_json_path, 'w') as fo:
    fo.write(graph)

# 下面的函数中我们导出神经网络模型的权重参数
param_path = "../models/yolov4_pc.params"
with open(param_path, 'wb') as fo:
    fo.write(relay.save_param_dict(params))
# -------------至此导出模型阶段已经结束--------

OpNotImplemented: The following operators are not supported for frontend ONNX: Softplus